In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
from pathlib import Path
import itertools

In [ ]:
for file in Path("graphs").glob("**/*.png"):
    file.unlink()

In [ ]:
dir = Path("benchmark_matrix")
with open(dir / "Graal_11_10_0_19.json") as f:
    graal = json.loads(f.read())["matrix"]
with open(dir / "HotSpot_11_10_0_19.json") as f:
    hotspot = json.loads(f.read())["matrix"]

In [ ]:
def plot_matrix(key, d, file_title, y_label, y_limit, layout=None):
    x_labels = list(d.keys())
    x = np.arange(len(x_labels))
    fig, ax = plt.subplots(figsize=(20, 10), layout=layout, dpi=300)
    
    gcs = sorted(graal[x_labels[0]])
    #def plot_sub_graph(ax, key, d, gcs, title, y_label, y_limit):
    width = 0.3
    multiplier = 0
    for gc in gcs:
        values = [value[gc][key] for value in d.values()]
        offset = width * multiplier
        rects = ax.bar(x + offset, values, width, label=gc)
        ax.bar_label(rects)
        multiplier += 1
        ## Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel(y_label)
        ax.set_xlabel("Heap Sizes (MB)")
        #ax.set_title(title)
        ax.set_xticks(x + width, x_labels)
        ax.legend(ncols=3, fontsize=20)
        ax.set_ylim(0, y_limit)
        fig.savefig(f"graphs/{file_title}.png")
            

In [ ]:
plot_matrix("throughput", graal,  "throughput_graalvm", "Throughput", 2, "tight")
plot_matrix("throughput", hotspot, "throughput_hotspot", "Throughput", 2, "tight")
plot_matrix("pause_time", graal, "pause_time_graalvm", "Pause Time", 2, "tight")
plot_matrix("pause_time", hotspot,  "pause_time_hotspot", "Pause Time",2.6, "tight" )

In [ ]:
def load_benchmarks(jdk, benchmark_group):
    benchmarks = {}
    files = [f for f in Path(".").glob("benchmark_stats/*.json") if jdk in f.name and benchmark_group in f.name and "error" not in f.name]
    for file in files:
        with open(file) as f:
            benchmark = json.loads(f.read())
            heap_size = benchmark["heap_size"]
            name = benchmark["benchmark_name"]
            gc = benchmark["garbage_collector"]
            throughput =  benchmark["throughput"]
            pause_time =  benchmark["p90_pause_time"]
            a = benchmarks.setdefault(heap_size, {}).setdefault(name, {}).setdefault(gc, {})
            a["throughput"] = throughput / 1e9
            a["pause_time"] = pause_time / 1e9
            
    # filter results not valid for all 3 GCs
    #print("Before", str(benchmarks)[:500])
    for key, value in list(benchmarks.items()):
        for k, v in list(value.items()):
           #print(f"Key={k} Value={v}")
           if len(v) != 3 or k == "spring": # filter spring
               #print(f"Deleting with keys: {key},{k}: {benchmarks[key][k]}")
               del benchmarks[key][k]
               #print(f"After delete with keys: {key},{k}: {benchmarks[key].get(k)}")
           #else:
           #    benchmarks[key] = dict(sorted(value.items()))
    for key in list(benchmarks):
        #print(f"Before: {benchmarks[key]}")
        benchmarks[key] = dict(sorted(benchmarks[key].items()))
        #print(f"After: {benchmarks[key]}")

    # assert everything is sorted
    # heap_size : name: gc
    for key in benchmarks:
        assert list(benchmarks[key].keys()) == sorted(list(benchmarks[key].keys())), f"Error: {list(benchmarks[key].keys())=} != {sorted(list(benchmarks[key].keys()))=}"
        #print(benchmarks[key].keys())
        for name in benchmarks[key]:
            assert len(benchmarks[key][name]) == 3, f"Error, {benchmarks[key][name]}"
        #a = [f"{benchmarks[key]} {v=} {len(v)=}" for v in benchmarks[key].values()]
        #print(a)
    #assert len(benchmarks['256']['kafka']) == 3, "hmm"
    #print(benchmarks['256']['kafka'])
    #print(benchmarks.keys())
    
        
    return benchmarks

In [ ]:

a = load_benchmarks("HotSpot", "DaCapo")
#for gc in ["Parallel", "G1", "Z"]:
#for key1 in a:
#    assert len(a[key1]) == 3

#print(a)
#print(a['512'].keys())
#print(a['512'].keys())
#print(a['256']['tradesoap']['Parallel'])

In [ ]:
def plot_benchmarks(benchmarks, key, y_label, file_title, layout=None, y_limit=None):
    x_labels = list(benchmarks.keys()) # This should be already sorted
    print(f"{x_labels=}")
    x = np.arange(len(x_labels))
    fig, ax = plt.subplots(figsize=(20, 10), layout=layout, dpi=300)

    gcs = sorted(benchmarks[x_labels[0]])
    width = 0.3
    multiplier = 0
    for gc in gcs:
        values = [value[gc][key] for value in benchmarks.values()]
        offset = width * multiplier
        rects = ax.bar(x + offset, values, width, label=gc)
        multiplier += 1
        ax.set_ylabel(y_label)
        ax.set_xlabel("Benchmarks")
        ax.set_xticks(x + width, x_labels)
        plt.xticks(rotation=45)
        ax.legend(ncols=3)
        ax.set_ylim(0, y_limit)
        #ax.set_title(file_title)
        #ax.bar_label(rects, label_type='center')
        fig.savefig(f"graphs/{file_title}.png")
        
def find_increment(number):
    n = str(number).split(".")
    num = 1
    if n[0] == "0":
        num = 0.1
        for i in n[1]:
            num *= 0.1
            if i != "0":
                break
    return num

    
#plt.tight_layout()
plt.rcParams.update({'font.size': 22})
for runtime, group in itertools.product(["Graal", "HotSpot"], ["Renaissance", "DaCapo"]):
    benchmarks = load_benchmarks(runtime, group)
    for heap_size in benchmarks:
        plt.close("all")
        max_throughput = max([v2["throughput"] for v1 in benchmarks[heap_size].values() for v2 in v1.values()])
        max_latency = max([v2["pause_time"] for v1 in benchmarks[heap_size].values() for v2 in v1.values()])
        #print("Antes", max_throughput, max_latency)
        max_latency += find_increment(max_latency)
        max_throughput += find_increment(max_throughput)
        #print("Depois", max_throughput, max_latency)
        plot_benchmarks(benchmarks[heap_size], "throughput", "Throughput (seconds)", f"{runtime}/{group}/{runtime}-{group}-{heap_size}-throughput", layout='tight',y_limit=max_throughput)
        plot_benchmarks(benchmarks[heap_size], "pause_time", "Pause Time (seconds)", f"{runtime}/{group}/{runtime}-{group}-{heap_size}-pause_time", layout='tight',y_limit=max_latency)

In [ ]:
def find_only_success():
    benchmarks = {}
    for file in Path("benchmark_stats/").glob("Renaissance*256*Graal*.json"):
        #print(file)
        with open(file) as f: 
            benchmark = json.loads(f.read())
        key = "error" if "error" in f"{file}" else "success"
        assert benchmark["benchmark_name"] is not None
        benchmarks.setdefault(benchmark["benchmark_name"], {}).setdefault(key, []).append(benchmark["garbage_collector"])
    import pprint
    for key in benchmarks:
        if len(benchmarks[key].get("success", {})) == 1:
            print(key)
find_only_success()

In [ ]:
def compute_benchmarks_len_report():
    benchmarks = {}
    for r in ["Graal", "HotSpot"]:
        for group in ["DaCapo", "Renaissance"]:
            for file in Path("benchmark_stats/").glob(f"{group}*{r}*.json"):
                with open(file) as f: 
                    b = json.loads(f.read())
                    gc = b["garbage_collector"]
                    name = b["benchmark_name"]
                    if name == "spring":
                        print("Skipping")
                        continue
                    status = "error" if bool(b["error"]) else "success"
                    hs = b["heap_size"]
                    entry = benchmarks.setdefault(r, {}).setdefault(hs, {}).setdefault(gc, {}).setdefault(status, [])
                    entry.append(name)
    nums = {}
    for r, h_dict in benchmarks.items():
        for heap, gc_dict in h_dict.items():
            for gc in gc_dict:
                error = gc_dict[gc].get("error", [])
                success = gc_dict[gc].get("success", [])
                error = len(error)
                success = len(success)
                gc_dict[gc]["error"] = error
                gc_dict[gc]["success"] = success
                nums.setdefault(heap,[]).append(error + success)
                
    for key in nums:
        print(f"{nums[key]=}")
        assert len(set(nums[key])) == 1
    
    with open("benchmarks_len_report.json", "w") as f:
        f.write(json.dumps(benchmarks))
    print(benchmarks)
compute_benchmarks_len_report()
    